## toy notebook for v2 pipeline on KF 1.7.0 manifests
* quick start v2 https://www.kubeflow.org/docs/components/pipelines/v2/pipelines/pipeline-basics/
* migration guide https://www.kubeflow.org/docs/components/pipelines/v2/migration/


In [1]:
import sys

In [2]:
# KFP sdk 2.0.1 works with Kubeflow Pipeline v2.0.0-beta.2 onwards
# KF 1.7.0 manifests has kfp server 2.0.0a7
# KFP sdk 2.0.0b13 works with Kubeflow Pipeline 2.0.0a6

# !{sys.executable} -m pip install --user --upgrade kfp==2.0.0b13

In [3]:
# !{sys.executable} -m pip uninstall -y kfp-server-api
# !{sys.executable} -m pip install --user --upgrade kfp-server-api==2.0.0
# !{sys.executable} -m pip install --user --upgrade kfp-pipeline-spec==0.2.2

In [4]:
!{sys.executable} -m pip list | grep kfp

kfp                      2.0.0b13
kfp-pipeline-spec        0.2.0
kfp-server-api           2.0.0a6


## restart kernel

In [5]:
from kfp import dsl
from kfp import compiler
# from kfp.components.pipeline_task import PipelineTask

@dsl.component
def square(x: float) -> float:
    # need to cast to float otherwise will run into an error
    return float(x ** 2)

## use kubectl -n kubeflow-kindfor logs pythagorean-tr97s-2385800675 to show the error log from console
@dsl.component
def add(x: float, z: float) -> float:
    # for some strange reason, the y param name is changed to "true", if i rename to z, it works
    return float(x + z)

@dsl.component
def square_root(x: float) -> float:
    return float(x ** .5)

# @dsl.pipeline(name='pythagorean-theorem-pipeline',
#               description='Solve for the length of a hypotenuse of a triangle with sides length `a` and `b`.',
#               pipeline_root='gs://my-pipelines-bucket',
#               display_name='Pythagorean pipeline.')
@dsl.pipeline
def pythagorean(a: float, b: float) -> float:
    a_sq_task = square(x=a)
    # if the limit is set to small, will be OOMKilled by kubernetes
    a_sq_task = a_sq_task.set_cpu_limit('1').set_memory_limit('500M')
    b_sq_task = square(x=b)
    b_sq_task = b_sq_task.set_cpu_limit('1').set_memory_limit('500M')
    # print(type(a_sq_task))
    sum_task = add(x=a_sq_task.output, z=b_sq_task.output)
    sum_task = sum_task.set_cpu_limit('1').set_memory_limit('500M')

    result_task = square_root(x=sum_task.output)
    result_task = result_task.set_cpu_limit('1').set_memory_limit('500M')
    return result_task.output

In [6]:
compiler.Compiler().compile(
    pipeline_func=pythagorean,
    package_path="./pythagorean.yaml"
)

In [7]:
from kfp.client import Client
client = Client()
NAMESPACE = client.get_user_namespace()
print(NAMESPACE)

kubeflow-kindfor


/home/jovyan/.local/lib/python3.8/site-packages/kfp/client/client.py:141: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-alpha.0 and later versions.
  warnings.warn(


In [8]:
client.list_experiments(namespace=NAMESPACE)

{'experiments': [{'created_at': datetime.datetime(2023, 6, 15, 16, 36, 7, tzinfo=tzlocal()),
                  'description': None,
                  'id': '44688294-d176-4974-a329-d5085422bf76',
                  'name': 'demo',
                  'resource_references': [{'key': {'id': 'kubeflow-kindfor',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'}],
 'next_page_token': None,
 'total_size': 1}

In [9]:
run = client.create_run_from_pipeline_func(
    pipeline_func=pythagorean,
    arguments = {"a": 1, "b": 2},
    run_name="my test v2 run",
    experiment_name = "demo",
    namespace=NAMESPACE,
    enable_caching=True,
    # enable_caching=False,
)